In [171]:
import numpy as np
import pandas as pd
import matplotlib as plt
import geopandas as gpd # conda install geopandas
import shapely
import folium # conda install -c conda-forge folium
import json
import pickle
import geopy
from geopy.extra.rate_limiter import RateLimiter
import re
import requests

### Import data
We import the dataset containing the food inspection and drop all irrelevant and null columns.

In [106]:
data = pd.read_csv('data/food-inspections.csv').drop(['AKA Name', 'City', 'State', "Historical Wards 2003-2015", "Zip Codes", "Community Areas", "Census Tracts", "Wards", "Location"], axis=1)
data.head()

,Inspection ID,DBA Name,License #,Facility Type,Risk,Address,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude
0,2320509,TAQUERIA EL DORADO,2694960.0,Restaurant,Risk 1 (High),2114 W LAWRENCE AVE,60625.0,2019-10-25T00:00:00.000,License Re-Inspection,Fail,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.968820,-87.682292
1,2320519,SALAM RESTAURANT,2002822.0,Restaurant,Risk 1 (High),4634-4636 N KEDZIE AVE,60625.0,2019-10-25T00:00:00.000,Complaint Re-Inspection,Pass,NaN,41.965719,-87.708538
2,2320421,THE NEW GRACE RESTAURANT,2698310.0,Restaurant,Risk 1 (High),4409 N BROADWAY,60640.0,2019-10-24T00:00:00.000,License,Not Ready,NaN,41.962104,-87.655204
3,2320368,GADS HILL CHILD CARE CENTER,2215799.0,Children's Services Facility,Risk 1 (High),2653 W OGDEN AVE,60608.0,2019-10-24T00:00:00.000,License,Pass,10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...,41.862273,-87.692703
4,2320389,"ALL THINGS ARE POSSIBLE FOR KIDS, INC.",2215965.0,Children's Services Facility,Risk 1 (High),4014 W CHICAGO AVE,60651.0,2019-10-24T00:00:00.000,License,Pass,54. GARBAGE & REFUSE PROPERLY DISPOSED; FACILI...,41.895468,-87.726665


# Complete the dataset

## Duplicates
Entries must be identified by their Inspection ID. Duplicates can simply be removed.

In [108]:
print("Number of duplicate rows: " + str(len(data[data.duplicated("Inspection ID")])))
data = data.drop_duplicates("Inspection ID")

Number of duplicate rows: 0


## Deal with missing values

In [109]:
for col in data.columns:
    print("Number of entries without " + col + ":" + str(len(data[data[col].apply(pd.isnull)])))

Number of entries without Inspection ID:0
Number of entries without DBA Name:0
Number of entries without License #:17
Number of entries without Facility Type:4770
Number of entries without Risk:72
Number of entries without Address:0
Number of entries without Zip:50
Number of entries without Inspection Date:0
Number of entries without Inspection Type:1
Number of entries without Results:0
Number of entries without Violations:51636
Number of entries without Latitude:682
Number of entries without Longitude:682


### License number

In [110]:
data[data['License #'].apply(pd.isnull)].sample(10)

,Inspection ID,DBA Name,License #,Facility Type,Risk,Address,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude
156097,670328,GOD'S BATTLE AXE PRAYER ACADEMY,NaN,CHURCH/DAY CARE,Risk 1 (High),6969 N WESTERN AVE,60645.0,2012-02-09T00:00:00.000,Canvass,Fail,"11. ADEQUATE NUMBER, CONVENIENT, ACCESSIBLE, D...",42.008303,-87.690005
151045,1184330,OLD ST. PATRICK'S CHURCH,NaN,Church,Risk 2 (Medium),700 W ADAMS ST,60661.0,2012-05-22T00:00:00.000,Special Events (Festivals),Pass,"34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOO...",41.879367,-87.644105
154371,1214242,GOD'S BATTLE AXE PRAYER ACADEMY,NaN,CHURCH/DAY CARE,Risk 1 (High),6969 N WESTERN AVE,60645.0,2012-03-19T00:00:00.000,Canvass Re-Inspection,Pass,NaN,42.008303,-87.690005
110647,1459918,ST. DEMETRIOS GREEK ORTHODOX CHURCH,NaN,Special Event,Risk 2 (Medium),2727 W WINONA ST,60625.0,2014-05-20T00:00:00.000,Canvass,Pass,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,41.974653,-87.697529
39837,2071910,ST. DEMETRIOS GREEK ORTHODOX CHURCH,NaN,Special Event,Risk 2 (Medium),2727 W WINONA ST,60625.0,2017-08-04T00:00:00.000,Canvass,Pass,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,41.974653,-87.697529
6403,2290863,ST. DEMETRIOS GREEK ORTHODOX CHURCH,NaN,Special Event,Risk 2 (Medium),2727 W WINONA ST,60625.0,2019-06-04T00:00:00.000,Canvass,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.974653,-87.697529
127949,1099104,ST. DEMETRIOS GREEK ORTHODOX CHURCH,NaN,Special Event,Risk 2 (Medium),2727 W WINONA ST,60625.0,2013-07-24T00:00:00.000,Canvass,Pass,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,41.974653,-87.697529
85255,1561809,ST. DEMETRIOS GREEK ORTHODOX CHURCH,NaN,Special Event,Risk 2 (Medium),2727 W WINONA ST,60625.0,2015-08-04T00:00:00.000,Canvass,Pass,NaN,41.974653,-87.697529
154583,679787,GOD'S BATTLE AXE PRAYER ACADEMY,NaN,CHURCH/DAY CARE,Risk 1 (High),6969 N WESTERN AVE,60645.0,2012-03-14T00:00:00.000,Canvass Re-Inspection,Fail,"9. WATER SOURCE: SAFE, HOT & COLD UNDER CITY P...",42.008303,-87.690005
65373,1933084,ST. DEMETRIOS GREEK ORTHODOX CHURCH,NaN,Special Event,Risk 2 (Medium),2727 W WINONA ST,60625.0,2016-06-20T00:00:00.000,Canvass,Pass,38. VENTILATION: ROOMS AND EQUIPMENT VENTED AS...,41.974653,-87.697529


### Facility Type

In [68]:
data[data['Facility Type'].apply(pd.isnull)].sample(10)

,Inspection ID,DBA Name,License #,Facility Type,Risk,Address,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude
181413,413970,BONEY-BONES BBQ,1717032.0,NaN,Risk 3 (Low),5620 S ASHLAND AVE,60636.0,2010-09-28T00:00:00.000,Canvass,Out of Business,NaN,41.791549,-87.664668
174271,88692,PROVECHO,1868064.0,NaN,Risk 3 (Low),5527-5531 N MILWAUKEE AVE,60630.0,2011-02-25T00:00:00.000,Canvass,Out of Business,NaN,41.982350,-87.773660
188054,250230,ALAN'S GROCERY,1937255.0,NaN,Risk 2 (Medium),202 N CALIFORNIA AVE,60612.0,2010-05-24T00:00:00.000,Out of Business,Pass,NaN,41.884369,-87.696526
189714,63320,SPARRER SAUSAGE CO,4040.0,NaN,Risk 3 (Low),4325 W OGDEN AVE,60623.0,2010-04-22T00:00:00.000,Canvass,Fail,NaN,41.848356,-87.732128
179036,456367,LITTLE CAESAR PIZZA #1726,1915444.0,NaN,Risk 2 (Medium),609 E 79TH ST,60619.0,2010-11-08T00:00:00.000,Canvass,Out of Business,NaN,41.751097,-87.609683
181126,414077,EL COQUI RESTAURANT,1771894.0,NaN,Risk 1 (High),4612 S ASHLAND AVE,60609.0,2010-10-01T00:00:00.000,Canvass,Out of Business,NaN,41.810015,-87.665157
142412,1286155,STUDIO 69 CAFE,1847813.0,NaN,Risk 1 (High),2366-2370 N NEVA AVE,60707.0,2012-10-26T00:00:00.000,Canvass,Out of Business,NaN,41.923010,-87.804536
119773,1391290,MINI MART,1576046.0,NaN,Risk 3 (Low),414 N CLARK ST,60610.0,2013-11-27T00:00:00.000,Canvass,Out of Business,NaN,41.889760,-87.631221
76905,1591435,YOU-IN-WIRELESS,1517083.0,NaN,Risk 3 (Low),328 E 43RD ST,60653.0,2015-12-09T00:00:00.000,Canvass,Out of Business,NaN,41.816718,-87.618511
100852,1447538,I DREAM OF FALAFEL,2032750.0,NaN,Risk 1 (High),329 S FRANKLIN ST,NaN,2014-10-23T00:00:00.000,Canvass,Business Not Located,NaN,41.877397,-87.635043


### Risk

In [69]:
data[data['Risk'].apply(pd.isnull)].sample(10)

,Inspection ID,DBA Name,License #,Facility Type,Risk,Address,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude
155833,670575,RESPECT FOR LIFE,0.0,Restaurant,NaN,725 W 79TH ST,60620.0,2012-02-16T00:00:00.000,Pre-License Consultation,Fail,NaN,41.750553,-87.642674
159299,537746,SOUTH SHORE TOURISUM CENTER,0.0,NaN,NaN,1813 E 71ST ST,60649.0,2011-12-02T00:00:00.000,Complaint,Business Not Located,NaN,41.765920,-87.580767
69313,1763609,THE INDIAN GARDEN RESTAURANT,2458737.0,NaN,NaN,700 E GRAND AVE,60611.0,2016-04-22T00:00:00.000,License,Not Ready,NaN,41.892249,-87.609518
177961,469702,NICOLES SWEET SWEET TOUTH,0.0,NaN,NaN,11946 S HALSTED ST,60628.0,2010-12-02T00:00:00.000,Complaint,Fail,NaN,41.676421,-87.642008
158082,659843,LAS BRISAS DEL SUR INC,2134071.0,Mobile Food Dispenser,NaN,2637 S THROOP ST FL,60608.0,2012-01-03T00:00:00.000,License,Fail,NaN,41.844671,-87.654962
17831,2229372,RICE THAI CAFE,2589016.0,NaN,NaN,211`7 W FARRAGUT AVE,60625.0,2018-10-04T00:00:00.000,License,Business Not Located,NaN,NaN,NaN
1990,2311745,MOJO 33 NORTH LASALLE LLC,2689550.0,Restaurant,NaN,33 N LA SALLE ST,60602.0,2019-09-04T00:00:00.000,License,Not Ready,NaN,41.882798,-87.632242
192096,58348,UNIVERSITY OF AESTHETICS/TOUCH CLINIC,0.0,NaN,NaN,1357 N MILWAUKEE AVE,60622.0,2010-03-04T00:00:00.000,Short Form Complaint,Pass,NaN,41.906596,-87.671323
7935,2288271,TAVERN ON THE POINT,2652668.0,NaN,NaN,6722-26 N NORTHWEST HWY,60631.0,2019-05-06T00:00:00.000,License,Not Ready,NaN,42.003187,-87.817026
179474,428391,BEST SUBS,0.0,NaN,NaN,2653 N WASTENAW AVE BLDG,60647.0,2010-10-29T00:00:00.000,Complaint,Business Not Located,NaN,NaN,NaN


### Zip

In [70]:
data[data['Zip'].apply(pd.isnull)].sample(10)

,Inspection ID,DBA Name,License #,Facility Type,Risk,Address,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude
84837,1562130,PARKWEST LIQUORS AND SMOKE SHOP,2354157.0,Liquor,Risk 3 (Low),2570 N LINCOLN AVE,NaN,2015-08-12T00:00:00.000,License,Fail,12. HAND WASHING FACILITIES: WITH SOAP AND SAN...,41.928597,-87.653388
174650,525229,GOLDEN HOUSE RESTAURANT,1443483.0,Restaurant,Risk 1 (High),4742-44 N RACINE AVE,NaN,2011-02-17T00:00:00.000,Canvass Re-Inspection,Pass,"30. FOOD IN ORIGINAL CONTAINER, PROPERLY LABEL...",41.968491,-87.659816
113626,1464217,DUNKIN DONUTS,1515116.0,Restaurant,Risk 2 (Medium),7545 N PAULINA ST,NaN,2014-04-02T00:00:00.000,Canvass,Out of Business,NaN,42.019032,-87.673459
30229,2135962,PERSPECTIVES CHARTER SCHOOL,2225546.0,School,Risk 1 (High),1930 S ARCHER AVE,NaN,2018-01-25T00:00:00.000,Canvass,Out of Business,NaN,41.856033,-87.628528
182422,413369,I DREAM OF FALAFEL,2032750.0,NaN,Risk 1 (High),329 S FRANKLIN ST,NaN,2010-09-13T00:00:00.000,License,Business Not Located,NaN,41.877397,-87.635043
139613,1279336,UIC COLLEGE PREP,2225460.0,School,Risk 1 (High),1231 S DAMEN,NaN,2012-12-18T00:00:00.000,Canvass,Pass,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,41.865800,-87.676074
100852,1447538,I DREAM OF FALAFEL,2032750.0,NaN,Risk 1 (High),329 S FRANKLIN ST,NaN,2014-10-23T00:00:00.000,Canvass,Business Not Located,NaN,41.877397,-87.635043
174875,525226,THE LUNCH MACHINE INC,2079077.0,Mobile Food Dispenser,Risk 2 (Medium),1319 W WILSON AVE,NaN,2011-02-15T00:00:00.000,License,Fail,NaN,41.965212,-87.663260
124446,1115029,SPORT SERVICE SOLDIER FIELD,1354323.0,Restaurant,Risk 2 (Medium),1410 S MUSEUM CAMPUS DR.,NaN,2013-09-19T00:00:00.000,Canvass,Out of Business,NaN,41.862628,-87.615031
174129,525238,THE LUNCH MACHINE INC,2079077.0,Mobile Food Dispenser,Risk 2 (Medium),1319 W WILSON AVE,NaN,2011-03-01T00:00:00.000,License Re-Inspection,Fail,NaN,41.965212,-87.663260


### Inspection Type

In [71]:
data[data['Inspection Type'].apply(pd.isnull)]

,Inspection ID,DBA Name,License #,Facility Type,Risk,Address,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude
63843,1946612,ARAMARL-FRESHII,2470318.0,Restaurant,Risk 1 (High),2650 N CLARK ST,60614.0,2016-07-14T00:00:00.000,NaN,Pass,NaN,41.930833,-87.644135


### Violations

In [164]:
data[data['Violations'].apply(pd.isnull)].sample(10)
data['Violations'] = data['Violations'].fillna('No data')

### Latitude and longitude
There are entries that are missing coordinate data, which is needed to put them on the map.
We see that all entries have address data, so we try to fill in the null values for the coordinates by using Geopy that uses geocoders to get coordinates based on addresses. Through testing we found that using ArcGIS as a geocoder gives us a sufficiently good result.

In [76]:
locator = geopy.geocoders.ArcGIS(user_agent='myGeocoder')
data_wo_latlong = data[data['Latitude'].apply(np.isnan)].groupby("DBA Name").first()
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
lats = pd.Series(index=data_wo_latlong.index)
longs = pd.Series(index=data_wo_latlong.index)
for name, address in list(zip(data_wo_latlong.index, data_wo_latlong['Address'])):
    if location == "":
        continue
    location = geocode(address + ", Chicago, United States")
    if location is not None:
        lats[name] = location.latitude
        longs[name] = location.longitude
    else:
        print('Did not find location for address: ' + address)

for name, lat, long in zip(lats.index, lats, longs):
    data['Latitude'] = np.where(data['DBA Name'] == name, lat, data['Latitude'])
    data['Longitude'] = np.where(data['DBA Name'] == name, long, data['Longitude'])


RateLimiter caught an error, retrying (0/2 tries). Called with (*('9513 S RIDGELAND AVE STE 3E, Chicago, United States',), **{}).
Traceback (most recent call last):
  File "/home/widmark/Anaconda3/lib/python3.7/site-packages/geopy/geocoders/base.py", line 355, in _call_geocoder
    page = requester(req, timeout=timeout, **kwargs)
  File "/home/widmark/Anaconda3/lib/python3.7/urllib/request.py", line 525, in open
    response = self._open(req, data)
  File "/home/widmark/Anaconda3/lib/python3.7/urllib/request.py", line 543, in _open
    '_open', req)
  File "/home/widmark/Anaconda3/lib/python3.7/urllib/request.py", line 503, in _call_chain
    result = func(*args)
  File "/home/widmark/Anaconda3/lib/python3.7/urllib/request.py", line 1360, in https_open
    context=self._context, check_hostname=self._check_hostname)
  File "/home/widmark/Anaconda3/lib/python3.7/urllib/request.py", line 1320, in do_open
    r = h.getresponse()
  File "/home/widmark/Anaconda3/lib/python3.7/http/client.py"

Did not find location for address: 9513 S RIDGELAND AVE STE 3E


In [77]:
print("Number of entries without coordinate data: " + str(len(data[data["Latitude"].apply(pd.isnull)])))

Number of entries without coordinate data: 1


# Projecting food inspections on a map
We want to be able to visualize our dataset on a map based on coordinates. This is done by using Folium.

In [84]:
data_names = data.groupby('DBA Name').first()
print(len(data_names))

27491


In [88]:
middle_lat = data_names.Latitude.median()
middle_lon = data_names.Longitude.median()
ch_map = folium.Map(location = [middle_lat, middle_lon], zoom_start=10)
locations = folium.map.FeatureGroup()
lats = data_names.Latitude
lons = data_names.Longitude
names = data_names.index


for lat, lon, name in list(zip(lats, lons, names))[0:100]:
    if np.isnan(lat) or np.isnan(lon):
        continue
    folium.Marker(
        location =  [lat, lon],
        tooltip = name,
        icon = folium.Icon(color='green', icon='info-sign')
    ).add_to(ch_map)
    
ch_map.add_child(locations)
ch_map

# Finding useful data in the Violations column
Violations seem to have the structure:
(&lt;violation number>. &lt;violation title> - &lt;violation comments> - | )+

In [165]:
data['Violations'][0]

"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL EMPLOYEE; KNOWLEDGE, RESPONSIBILITIES AND REPORTING - Comments: FOUND NO EMPLOYEE HEALTH POLICY ON SITE. INSTRUCTED TO PROVIDE AN EMPLOYEE HEALTH POLICY THAT INCLUDES A SIGNED ACKNOWLEDGEMENT FROM EACH EMPLOYEE. PRIORITY FOUNDATION VIOLATION 7-38-010,NO CITATION ISSUED. - - | 5. PROCEDURES FOR RESPONDING TO VOMITING AND DIARRHEAL EVENTS - Comments: : FOUND NO PROCEDURE/PLAN FOR RESPONDING TO VOMITING AND DIARRHEAL EVENTS AND NO CLEAN-UP ITEMS.MUST PROVIDE AND MAINTAIN APPROPRIATE SUPPLIES OR KIT. (NECESSARY ITEMS AT A MINIMUM: GLOVES, FACE MASKS, DISPOSABLE MOPS AND APPROPRIATE SANITIZER THAT KILLS NOROVIRUS).PRIORITY FOUNDATION VIOLATION 7-38-005,NO CITATION ISSUED. | 25. CONSUMER ADVISORY PROVIDED FOR RAW/UNDERCOOKED FOOD - Comments: FOUND NO CONSUMER ADVISORY DISCLOSURE AND REMINDER STATEMENT ON THE MENU. MANAGEMENT INSTRUCTED TO DISCLOSE WHICH FOODS CAN BE ORDERED 'RAW' OR UNDER COOKED AND LINK THOSE FOODS WITH AN ASTERISK (*) TO A REMI

In [167]:
re_num = re.compile(r'([0-9]+)\.')
violation_numbers = data['Violations'].apply(re_num.findall)
violation_numbers
# Maybe we can actually connect violation numbers to result "Out of business"
# See under "Under what conditions will a food establishment get shut down?" https://dchealth.dc.gov/service/understanding-food-establishment-inspections
# While this is for Washington DC, I think it could apply to us too

0                   [3, 5, 25, 50, 74, 53, 53, 56, 58]
1                                                   []
2                                                   []
3                                         [10, 55, 56]
4                                         [54, 55, 56]
                              ...                     
194899                                              []
194900                            [18, 32, 34, 35, 36]
194901                                              []
194902    [18, 19, 32, 33, 34, 35, 36, 37, 38, 39, 41]
194903                                              []
Name: Violations, Length: 194708, dtype: object

# Supplementing the data

## Income by zip code (actually, anything by zip code if we want)
https://towardsdatascience.com/getting-census-data-in-5-easy-steps-a08eeb63995d

Attribution statement: This product uses the Census Bureau Data API but is not endorsed or certified by the Census Bureau.

Zip code 60666 is the zip code for O'Hare International Airport.
Zip code 60627 is the zip code for Golden Gate Park.
These do not have a household median income since nobody lives there.

In [184]:
income_by_zip_address = "https://api.census.gov/data/%s/acs/acs5?key=%s&get=%s&for=zip%%20code%%20tabulation%%20area:%s" 
year = "2017" # 2009-2017
api_key = "" # Request here: https://www.census.gov/developers/
income_variable = "B19013_001E"
zip_codes = data['Zip'].drop_duplicates()

for zip_code in zip_codes:
    if pd.isnull(zip_code):
        continue
        
    called_api = income_by_zip_address % (year, api_key, income_variable, int(zip_code))
    response = requests.get(called_api)
    if response.status_code == 200:
        formatted_response = json.loads(response.text)
        print(formatted_response)
    else:
        print('Could not find income for zip code: ' + str(int(zip_code)))
    # add to some list or something, also exception handling
    

    

[['B19013_001E', 'zip code tabulation area'], ['63106', '60625']]
[['B19013_001E', 'zip code tabulation area'], ['50488', '60640']]
[['B19013_001E', 'zip code tabulation area'], ['41226', '60608']]
[['B19013_001E', 'zip code tabulation area'], ['36283', '60651']]
[['B19013_001E', 'zip code tabulation area'], ['68223', '60647']]
[['B19013_001E', 'zip code tabulation area'], ['67045', '60638']]
[['B19013_001E', 'zip code tabulation area'], ['96040', '60611']]


KeyboardInterrupt: 

# Information about columns

## Inspection types
From https://data.cityofchicago.org/api/assets/BAD5301B-681A-4202-9D25-51B2CAE672FF:
Inspection type: An inspection can be one of the following types: canvass, the most common type of inspection performed at a frequency relative to the risk of the establishment; consultation, when the inspection is done at the request of the owner prior to the opening of the establishment; complaint, when  the inspection is done in response to a complaint against the establishment; license, when the inspection is done as a requirement for the establishment to receive its license to operate; suspect food poisoning, when the inspection is done in response to one or more persons claiming to have gotten ill as a result of eating at the establishment (a specific type of complaint-based inspection); task-force inspection, when an inspection of a bar or tavern is done. Re-inspections can occur for most types of these inspections and are indicated as such. 

In [213]:
inspection_types = data['Inspection Type'].drop_duplicates().reset_index(drop=True).apply(str)
valid_inspection_types = ['canvass', 'consultation', 'license', 'food poisoning', 'task force']
print(inspection_types[inspection_types.apply(str.lower).map(lambda x: any([y in x for y in valid_inspection_types]))])
print(inspection_types[inspection_types.apply(str.lower).map(lambda x: not any([y in x for y in valid_inspection_types]))])

0                          License Re-Inspection
2                                        License
3                                        Canvass
4                          Canvass Re-Inspection
8                       Suspected Food Poisoning
9         Suspected Food Poisoning Re-inspection
11                                  Consultation
19                            License-Task Force
26                     LICENSE CANCELED BY OWNER
27             OWNER SUSPENDED OPERATION/LICENSE
28                          LICENSE CONSULTATION
29                          License consultation
30                        Task Force Liquor 1475
34                      Pre-License Consultation
35                        CANVASS SPECIAL EVENTS
36                  CANVASS SCHOOL/SPECIAL EVENT
40                        TASK FORCE LIQUOR 1470
42                             license task 1474
44                    Task Force for liquor 1474
46                     1315 license reinspection
47             Task 

In [168]:
# g_data = gpd.GeoDataFrame(data)
# g_data = g_data.set_geometry([shapely.geometry.Point(x, y) for x, y in zip(data.Longitude, data.Latitude)])
# g_data.head()

In [169]:
# chicago_map = gpd.read_file('data/chicago_zip.shp') # https://www.kaggle.com/threadid/chicago-shape-files

In [170]:
# ax = chicago_map.plot()
# g_data.plot(ax=ax, color='red')